# 1 Подготовка

## Импорт библиотек

In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import pickle
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import zipfile

## Устанавливаем стиль визуализации

In [2]:
sns.set_theme(style="whitegrid")

# 2 Загрузка данных

In [3]:
# Загрузка данных локально
file_path = '../data/tickets.csv'

data = pd.read_csv(file_path, delimiter=';')

In [ ]:
# Загрузка данных из внешнего хранилища

url = "https://www.kaggle.com/api/v1/datasets/download/adisongoh/it-service-ticket-classification-dataset"  # Адрес для загрузки данных

extract_file = "all_tickets_processed_improved_v3.csv"  # Имя файла в архиве

# Скачивание архива
response = requests.get(url, stream=True)
if response.status_code == 200:
    temp_archive = "../data/temp_archive.zip"
    with open(temp_archive, 'wb') as f:
        f.write(response.content)
    print(f"Данные успешно загружены с {url}")
else:
    raise Exception(f"Ошибка при загрузке данных: {response.status_code} {response.reason}")

# Распаковка архива
extract_dir = '../data'
with zipfile.ZipFile(temp_archive, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Архив успешно распакован.")
extracted_path = os.path.join(extract_dir, extract_file)

# Очистка временных файлов
os.remove(temp_archive)
print("Временный архив удалён.")

# Загрузка данных в DataFrame
data = pd.read_csv(extracted_path, delimiter=',')

# Переименование колонок при необходимости
rename_mapping = {
    'Document': 'Description',
    'Topic_group': 'Type'
}
data.rename(columns=rename_mapping, inplace=True)

In [ ]:
# Проверка корректной загрузки данных
print("Пример данных:")
display(data.head())

# 3 Исследовательский анализ данных

In [ ]:
# Распределение категорий тикетов
plt.figure(figsize=(10, 6))
sns.countplot(
    y=data['Type'], 
    order=data['Type'].value_counts().index, 
    palette="viridis", 
    legend=False
)
plt.title('Распределение категорий тикетов')
plt.xlabel('Количество')
plt.ylabel('Категория')
plt.show()

In [ ]:
# Длина описаний тикетов
data['description_length'] = data['Description'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(data['description_length'], kde=True, bins=30, color='blue')
plt.title('Распределение длины описаний тикетов')
plt.xlabel('Длина описания')
plt.ylabel('Количество')
plt.show()

# 4 Предобработка данных

In [7]:
# Удаляем строки с пропущенными значениями
data = data.dropna(subset=['Description', 'Type'])

In [8]:
# Приведение текста к нижнему регистру
data['Description'] = data['Description'].str.lower()

In [9]:
# Удаление лишних символов (шум)
data['Description'] = data['Description'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
# Выводим несколько обработанных записей
print("\nОбработанные данные:")
display(data.head())

# 5 Разделение на признаки и целевые переменные, векторизация данных TF-IDF

In [11]:
X = data['Description']  # Признаки
y = data['Type']         # Целевая переменная

## Создание обучающей и валидационной выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nКоличество записей в обучающей выборке: {len(X_train)}")
print(f"Количество записей в валидационной выборке: {len(X_test)}")

In [ ]:
# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\nРазмерность векторизованных данных:")
print(f"Обучающие данные: {X_train_tfidf.shape}")
print(f"Валидационные данные: {X_test_tfidf.shape}")

# 6 Обучение модели с использованием метода логистической регрессии

In [ ]:
model = LogisticRegression(random_state=42, max_iter=1000)

model.fit(X_train_tfidf, y_train)

# 7 Проверка работоспособности и оценка модели

In [ ]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


print("\nAccuracy модели:", accuracy)
print("\nОтчет по классификации:\n", classification_rep)

# 8 Сохранение модели

In [16]:
# Директория для сохранения модели
model_path = '../models/ticket_classifier.pkl'
vectorizer_path = '../models/tfidf_vectorizer.pkl'

# Сохранение модели
with open(model_path, 'wb') as model_file:
    pickle.dump(model, model_file)

# Сохранение векторизатора
with open(vectorizer_path, 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# 9 Загрузка и проверка модели

In [ ]:
# Загрузка модели
with open(model_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Загрузка векторизатора
with open(vectorizer_path, 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

print("\nМодель и векторизатор успешно загружены.")

In [ ]:
# Тестирование загруженной модели на новом тексте
new_ticket = ["Reports fail to load when certain filters are applied"]
new_ticket_vectorized = loaded_vectorizer.transform(new_ticket)
new_ticket_prediction = loaded_model.predict(new_ticket_vectorized)

print("\nТестовое описание тикета:", new_ticket[0])
print("Предсказанная категория:", new_ticket_prediction[0])